In [26]:
import pandas as pd
import numpy as np
import sqlite3
import time

In [2]:
trips = pd.read_csv('../../CSVs/trips.csv', low_memory=False)
stop_times = pd.read_csv('../../CSVs/stop_times.csv', low_memory=False)
#st_update = pd.read_csv('../../CSVs/stop_time_updates.csv', usecols=['stop_sequence', 'arrival_time', 'trip_update_id'], low_memory=False)
trp_update = pd.read_csv('../../CSVs/trip_updates.csv', usecols=['trip_id', 'route_id', 'timestamp', 'trip_start_time'], low_memory=False)

In [3]:
trp_update.head()

,trip_id,route_id,trip_start_time,timestamp
0,35242107,1,NaN,2017-09-25 22:33:24.000000
1,35242108,1,NaN,2017-09-25 22:33:24.000000
2,35242109,1,NaN,2017-09-25 22:33:24.000000
3,35242110,1,NaN,2017-09-25 22:33:24.000000
4,35242111,1,NaN,2017-09-25 22:33:24.000000


In [4]:
stop_times = stop_times.loc[:, ['trip_id', 'stop_id', 'stop_sequence', 'arrival_time', 'departure_time']]
stop_times.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
1,Logan-22-Weekday-trip,Logan-RentalCarCenter,2,08:04:00,08:04:00
2,Logan-22-Weekday-trip,Logan-A,3,08:09:00,08:09:00
3,Logan-22-Weekday-trip,Logan-B,4,08:12:00,08:12:00
4,Logan-22-Weekday-trip,Logan-Subway,5,08:17:00,08:17:00


In [5]:
trips = trips.loc[:, ['trip_id', 'route_id', 'service_id', 'trip_headsign']]
trips.head()

,trip_id,route_id,service_id,trip_headsign
0,Logan-22-Weekday-trip,Logan-22,Logan-Weekday,Loop
1,Logan-22-Weekend-trip,Logan-22,Logan-Weekend,Loop
2,Logan-33-Weekday-trip,Logan-33,Logan-Weekday,Loop
3,Logan-33-Weekend-trip,Logan-33,Logan-Weekend,Loop
4,Logan-55-Weekday-trip,Logan-55,Logan-Weekday,Loop


Obtain all the first stops for all trips as well as their expected arrival time.

In [6]:
first_stops = stop_times.loc[stop_times['stop_sequence'] == 1, :]
first_stops.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
7,Logan-22-Weekend-trip,Logan-Subway,1,12:00:00,12:00:00
14,Logan-33-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
21,Logan-33-Weekend-trip,Logan-Subway,1,12:00:00,12:00:00
28,Logan-55-Weekday-trip,Logan-Subway,1,00:00:00,00:00:00


In [14]:
[trp_update['timestamp'].min(),trp_update['timestamp'].max()]

['2017-09-25 22:33:24.000000', '2017-09-26 12:33:39.000000']

Above is the time the first update and last update were obtained. Now to find trips that are running around the same time real time data was being gathered.

In [16]:
first_stops[first_stops['arrival_time'] == '21:00:00'][:25]

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
1662,CR-Saturday-October-17-L-1314,Lowell,1,21:00:00,21:00:00
1790,CR-Saturday-August-17-L-1314,Lowell,1,21:00:00,21:00:00
3591,CR-Saturday-Spring-17-1160,Newburyport,1,21:00:00,21:00:00
4691,CR-Sunday-October-17-L-2314,Lowell,1,21:00:00,21:00:00
4819,CR-Sunday-August-17-L-2314,Lowell,1,21:00:00,21:00:00
6313,CR-Sunday-Spring-17-2160,Newburyport,1,21:00:00,21:00:00
8845,CR-Weekday-Spring-17-530,Worcester,1,21:00:00,21:00:00
9852,CR-Weekday-Spring-17-727,South Station,1,21:00:00,21:00:00
10267,CR-Weekday-Spring-17-784,Readville,1,21:00:00,21:00:00
10275,CR-Weekday-Spring-17-785,South Station,1,21:00:00,21:00:00


In [19]:
trp_update[trp_update['trip_id'] == '34663362'].head()

,trip_id,route_id,trip_start_time,timestamp
196427,34663362,Mattapan,NaN,2017-09-26 01:01:39.000000
197029,34663362,Mattapan,NaN,2017-09-26 01:02:13.000000
197620,34663362,Mattapan,NaN,2017-09-26 01:03:54.000000
198215,34663362,Mattapan,NaN,2017-09-26 01:04:27.000000
198809,34663362,Mattapan,NaN,2017-09-26 01:05:01.000000


In [22]:
stop_times[stop_times['trip_id'] == '34663362']

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
80125,34663362,70276,1,21:00:00,21:00:00
80126,34663362,70274,2,21:01:00,21:01:00
80127,34663362,70272,3,21:02:00,21:02:00
80128,34663362,70270,4,21:03:00,21:03:00
80129,34663362,70268,5,21:04:00,21:04:00
80130,34663362,70266,6,21:05:00,21:05:00
80131,34663362,70264,7,21:07:00,21:07:00
80132,34663362,70262,8,21:08:00,21:08:00


Above is the expected arrival times for trip 34663362

In [8]:
# connect to the sqlite database for querying data from stop time updates
conn = sqlite3.connect('../../gtfs.db')

In [23]:
first_update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 196427', conn)
first_update

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,4473568,1,70261,0,1506388335,0,0,1506388335,0,SCHEDULED,196427
1,4473569,2,70263,0,1506388393,0,0,1506388393,0,SCHEDULED,196427
2,4473570,3,70265,0,1506388503,0,0,1506388503,0,SCHEDULED,196427
3,4473571,4,70267,0,1506388584,0,0,1506388584,0,SCHEDULED,196427
4,4473572,5,70269,0,1506388671,0,0,1506388671,0,SCHEDULED,196427
5,4473573,6,70271,0,1506388779,0,0,1506388779,0,SCHEDULED,196427
6,4473574,7,70273,0,1506388856,0,0,1506388856,0,SCHEDULED,196427
7,4473575,8,70275,0,1506388946,0,0,1506388946,0,SCHEDULED,196427


We can see that for the first update we the predicted arrival times for all the stops in trip 34663362. Lets look at the predicted arrival times from the second update (trip_update_id = 197029)

In [24]:
second_update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 197029', conn)
second_update

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,4486918,1,70261,0,1506388339,0,0,1506388339,0,SCHEDULED,197029
1,4486919,2,70263,0,1506388397,0,0,1506388397,0,SCHEDULED,197029
2,4486920,3,70265,0,1506388507,0,0,1506388507,0,SCHEDULED,197029
3,4486921,4,70267,0,1506388588,0,0,1506388588,0,SCHEDULED,197029
4,4486922,5,70269,0,1506388675,0,0,1506388675,0,SCHEDULED,197029
5,4486923,6,70271,0,1506388783,0,0,1506388783,0,SCHEDULED,197029
6,4486924,7,70273,0,1506388860,0,0,1506388860,0,SCHEDULED,197029
7,4486925,8,70275,0,1506388950,0,0,1506388950,0,SCHEDULED,197029


On the second update we still have the predicted arrival times for all the stops. Lets compare the predicted arrival time from the first and second update. Their epoch times differ, but lets get a more human readable format.

In [27]:
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(first_update['arrival_time'][0]))

'2017-09-25 20:12:15'

In [28]:
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(second_update['arrival_time'][0]))

'2017-09-25 20:12:19'

The predicted arrival times for the first sequence don't differ by much just 4 seconds

In [33]:
third_update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 198809', conn)
third_update

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,4526145,1,70261,0,1506388296,0,0,1506388296,0,SCHEDULED,198809
1,4526146,2,70263,0,1506388354,0,0,1506388354,0,SCHEDULED,198809
2,4526147,3,70265,0,1506388464,0,0,1506388464,0,SCHEDULED,198809
3,4526148,4,70267,0,1506388545,0,0,1506388545,0,SCHEDULED,198809
4,4526149,5,70269,0,1506388632,0,0,1506388632,0,SCHEDULED,198809
5,4526150,6,70271,0,1506388740,0,0,1506388740,0,SCHEDULED,198809
6,4526151,7,70273,0,1506388817,0,0,1506388817,0,SCHEDULED,198809
7,4526152,8,70275,0,1506388907,0,0,1506388907,0,SCHEDULED,198809
